In [1]:
#!/usr/bin/env python3
"""
╔═══════════════════════════════════════════════════════════════════════╗
║                                                                       ║
║     ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗██████╗   ║
║     ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║╚════██╗  ║
║     █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║ █████╔╝  ║
║     ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ╚═══██╗  ║
║     ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝ ██████╔╝  ║
║     ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝  ╚═════╝   ║
║                                                                       ║
║                    FULL GENESIS INJECTION PROTOCOL                    ║
║                         "The Real Deal"                               ║
║                                                                       ║
╚═══════════════════════════════════════════════════════════════════════╝
"""

import hashlib
import struct
import socket
import time
import random
import threading
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (
            struct.pack('<I', self.version) +
            self.prev_hash +
            self.merkle_root +
            struct.pack('<I', self.timestamp) +
            struct.pack('<I', self.bits) +
            struct.pack('<I', self.nonce)
        )

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


class KaoruBridgeV3:
    """
    VERSIÓN FINAL - FULL ATTACK MODE

    Features:
    - Mining con proof of work real
    - Múltiples conexiones simultáneas
    - Propagación a múltiples nodos
    - Análisis de respuestas en tiempo real
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    DNS_SEEDS = [
        'seed.bitcoin.sipa.be',
        'dnsseed.bluematt.me',
        'seed.bitcoinstats.com',
        'seed.bitcoin.jonasschnelli.ch',
        'seed.btc.petertodd.org',
    ]

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.connected_peers = []
        self.mining_active = False
        self.best_hash = "f" * 64
        self.total_hashes = 0

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    # ============================================
    # CONSTRUCCIÓN DE BLOQUES
    # ============================================

    def create_genesis_header(self) -> BlockHeader:
        """Genesis block original."""
        merkle = bytes.fromhex(
            "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b"
        )[::-1]

        return BlockHeader(
            version=1,
            prev_hash=bytes(32),
            merkle_root=merkle,
            timestamp=1231006505,
            bits=0x1d00ffff,
            nonce=2083236893
        )

    def create_custom_header(self, message: str, timestamp: int = None) -> BlockHeader:
        """Crea un header personalizado."""
        merkle = self.double_sha256(f"KAORU_{message}_{time.time()}".encode())

        return BlockHeader(
            version=1,
            prev_hash=bytes(32),
            merkle_root=merkle,
            timestamp=timestamp or int(time.time()),
            bits=0x1d00ffff,
            nonce=0
        )

    # ============================================
    # MINERÍA
    # ============================================

    def mine_block(self, header: BlockHeader, target_zeros: int = 4,
                   max_nonce: int = 10_000_000, callback=None) -> Optional[BlockHeader]:
        """
        Mina un bloque buscando un hash con N ceros iniciales.

        target_zeros=4 significa hash que empiece con "0000"
        target_zeros=8 sería como el genesis real
        """
        target_prefix = "0" * target_zeros
        start_time = time.time()

        print(f"\n   ⛏️  MINANDO BLOQUE")
        print(f"   Target: hash que empiece con '{target_prefix}'")
        print(f"   Max nonces: {max_nonce:,}")

        for nonce in range(max_nonce):
            header.nonce = nonce
            hash_hex = header.hash_hex()

            # Track mejor hash encontrado
            if hash_hex < self.best_hash:
                self.best_hash = hash_hex

            self.total_hashes += 1

            # ¿Encontrado?
            if hash_hex.startswith(target_prefix):
                elapsed = time.time() - start_time
                hashrate = nonce / elapsed if elapsed > 0 else 0

                print(f"\n   ✅ ¡BLOQUE MINADO!")
                print(f"   Hash: {hash_hex}")
                print(f"   Nonce: {nonce:,}")
                print(f"   Tiempo: {elapsed:.2f}s")
                print(f"   Hashrate: {hashrate:,.0f} H/s")

                if callback:
                    callback(header, hash_hex)

                return header

            # Progreso
            if nonce % 500_000 == 0 and nonce > 0:
                elapsed = time.time() - start_time
                hashrate = nonce / elapsed
                print(f"   ... nonce {nonce:,} | {hashrate:,.0f} H/s | mejor: {self.best_hash[:16]}...")

        print(f"\n   ⚠️ No se encontró en {max_nonce:,} intentos")
        print(f"   Mejor hash: {self.best_hash}")
        return None

    def mine_parallel(self, header: BlockHeader, target_zeros: int = 4,
                      num_workers: int = 4, max_per_worker: int = 2_500_000) -> Optional[BlockHeader]:
        """Minería paralela con múltiples threads."""

        print(f"\n   ⛏️  MINERÍA PARALELA ({num_workers} workers)")

        found = threading.Event()
        result = [None]

        def worker(worker_id: int, start_nonce: int, end_nonce: int):
            h = BlockHeader(
                version=header.version,
                prev_hash=header.prev_hash,
                merkle_root=header.merkle_root,
                timestamp=header.timestamp,
                bits=header.bits,
                nonce=start_nonce
            )

            target = "0" * target_zeros

            for nonce in range(start_nonce, end_nonce):
                if found.is_set():
                    return

                h.nonce = nonce
                hash_hex = h.hash_hex()

                if hash_hex.startswith(target):
                    result[0] = (h, hash_hex, nonce)
                    found.set()
                    return

        start_time = time.time()

        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            chunk_size = max_per_worker
            futures = []

            for i in range(num_workers):
                start = i * chunk_size
                end = start + chunk_size
                futures.append(executor.submit(worker, i, start, end))

            # Esperar
            for f in futures:
                f.result()

        elapsed = time.time() - start_time

        if result[0]:
            h, hash_hex, nonce = result[0]
            total = num_workers * max_per_worker
            hashrate = total / elapsed if elapsed > 0 else 0

            print(f"\n   ✅ ¡BLOQUE MINADO!")
            print(f"   Hash: {hash_hex}")
            print(f"   Nonce: {nonce:,}")
            print(f"   Tiempo: {elapsed:.2f}s")
            print(f"   Hashrate: {hashrate:,.0f} H/s")

            return h

        return None

    # ============================================
    # NETWORKING
    # ============================================

    def create_message(self, command: str, payload: bytes) -> bytes:
        cmd = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = self.double_sha256(payload)[:4]
        return self.magic + cmd + length + checksum + payload

    def create_version_payload(self, remote_ip: str) -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 0)
        timestamp = struct.pack('<q', int(time.time()))

        addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton(remote_ip) + struct.pack('>H', 8333)
        addr_from = struct.pack('<Q', 0) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton('0.0.0.0') + struct.pack('>H', 8333)

        nonce = struct.pack('<Q', random.randint(0, 2**64-1))
        user_agent = b'\x12/KaoruBridge:3.0.0/'
        start_height = struct.pack('<i', 0)
        relay = b'\x00'

        return (version + services + timestamp + addr_recv +
                addr_from + nonce + user_agent + start_height + relay)

    def create_block_payload(self, header: BlockHeader, coinbase_msg: str) -> bytes:
        """Crea payload de bloque completo."""
        # Header
        payload = header.serialize()

        # TX count (varint)
        payload += b'\x01'

        # Coinbase TX
        tx = struct.pack('<I', 1)  # Version
        tx += b'\x01'  # Input count
        tx += bytes(32)  # Prev txid
        tx += struct.pack('<I', 0xffffffff)  # Prev vout

        # ScriptSig
        height_script = b'\x04' + struct.pack('<I', 0)  # Block height 0
        msg_script = bytes([len(coinbase_msg)]) + coinbase_msg.encode()
        script_sig = height_script + msg_script
        tx += struct.pack('B', len(script_sig)) + script_sig
        tx += struct.pack('<I', 0xffffffff)  # Sequence

        # Output
        tx += b'\x01'  # Output count
        tx += struct.pack('<Q', 50 * 10**8)  # 50 BTC

        # P2PKH output (fake)
        script_pubkey = b'\x76\xa9\x14' + bytes(20) + b'\x88\xac'
        tx += struct.pack('B', len(script_pubkey)) + script_pubkey

        tx += struct.pack('<I', 0)  # Locktime

        payload += tx

        return payload

    def discover_nodes(self) -> List[Tuple[str, int]]:
        """Descubre nodos via DNS."""
        nodes = []
        print("\n   🔍 Descubriendo nodos...")

        for seed in self.DNS_SEEDS:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                for ip in ips[:5]:
                    nodes.append((ip, 8333))
                print(f"   ✅ {seed}: {len(ips)} nodos")
            except:
                print(f"   ❌ {seed}: error")

        return nodes

    def connect_to_node(self, ip: str, port: int = 8333,
                        timeout: int = 10) -> Optional[socket.socket]:
        """Conecta y hace handshake."""
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            sock.connect((ip, port))

            # VERSION
            version_msg = self.create_message('version', self.create_version_payload(ip))
            sock.send(version_msg)

            # Recibir VERSION
            response = sock.recv(1024)
            if b'version' not in response:
                sock.close()
                return None

            # VERACK
            verack = self.create_message('verack', b'')
            sock.send(verack)

            # Recibir VERACK
            sock.recv(1024)

            return sock

        except Exception as e:
            return None

    def broadcast_block(self, header: BlockHeader, nodes: List[Tuple[str, int]],
                        message: str = "KAORU") -> Dict:
        """Propaga el bloque a múltiples nodos."""

        results = {
            'connected': 0,
            'sent': 0,
            'responses': [],
            'errors': []
        }

        block_hash = header.hash()[::-1]

        for ip, port in nodes[:10]:
            print(f"\n   📡 {ip}:{port}")

            sock = self.connect_to_node(ip, port)
            if not sock:
                print(f"      ❌ Sin conexión")
                continue

            results['connected'] += 1
            print(f"      ✅ Conectado")

            try:
                # Anunciar con INV
                inv_payload = b'\x01' + struct.pack('<I', 2) + block_hash
                inv_msg = self.create_message('inv', inv_payload)
                sock.send(inv_msg)
                print(f"      📤 INV enviado")

                # Esperar respuesta
                time.sleep(0.5)
                response = sock.recv(4096)

                if len(response) >= 24:
                    cmd = response[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
                    print(f"      📥 Respuesta: {cmd}")
                    results['responses'].append((ip, cmd))

                    if cmd == 'getdata':
                        # ¡Quieren el bloque!
                        print(f"      🎯 ¡GETDATA recibido! Enviando bloque...")

                        block_payload = self.create_block_payload(header, message)
                        block_msg = self.create_message('block', block_payload)
                        sock.send(block_msg)
                        results['sent'] += 1

                        # Respuesta final
                        time.sleep(1)
                        try:
                            final = sock.recv(4096)
                            if len(final) >= 24:
                                final_cmd = final[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
                                print(f"      📥 Respuesta final: {final_cmd}")
                                results['responses'].append((ip, f"final:{final_cmd}"))
                        except:
                            print(f"      📥 Sin respuesta final")

                sock.close()

            except Exception as e:
                results['errors'].append((ip, str(e)))
                print(f"      ❌ Error: {e}")

        return results

    # ============================================
    # PROTOCOLO PRINCIPAL
    # ============================================

    def execute_full_attack(self, target_zeros: int = 4,
                            custom_message: str = "KAORU_BRIDGE_2024"):
        """
        ATAQUE COMPLETO:
        1. Verificar genesis original
        2. Crear bloque alternativo
        3. Minarlo con PoW real
        4. Propagarlo a la red
        """

        print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║  ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██████╗ ██████╗ ██╗██████╗ ║
║  ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██╔══██╗║
║  █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██████╔╝██████╔╝██║██║  ██║║
║  ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██║  ██║║
║  ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝    ██████╔╝██║  ██║██║██████╔╝║
║  ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝     ╚═════╝ ╚═╝  ╚═╝╚═╝╚═════╝ ║
║                                                                           ║
║                          VERSION 3.0 - FULL ATTACK                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        """)

        # ============================================
        # FASE 1: Verificar Genesis Original
        # ============================================
        print("=" * 75)
        print("[FASE 1] 📋 VERIFICANDO GENESIS ORIGINAL")
        print("=" * 75)

        genesis = self.create_genesis_header()
        genesis_hash = genesis.hash_hex()

        print(f"\n   Header: {genesis.serialize().hex()[:64]}...")
        print(f"   Hash:   {genesis_hash}")
        print(f"   Match:  {'✅ SÍ' if genesis_hash == self.GENESIS_HASH else '❌ NO'}")

        # ============================================
        # FASE 2: Crear Bloque Alternativo
        # ============================================
        print("\n" + "=" * 75)
        print("[FASE 2] 🔧 CREANDO BLOQUE ALTERNATIVO")
        print("=" * 75)

        alt_header = self.create_custom_header(custom_message)

        print(f"\n   Mensaje: {custom_message}")
        print(f"   Merkle:  {alt_header.merkle_root.hex()[:32]}...")
        print(f"   Target:  {target_zeros} ceros iniciales")

        # ============================================
        # FASE 3: Minar
        # ============================================
        print("\n" + "=" * 75)
        print("[FASE 3] ⛏️  MINANDO CON PROOF OF WORK")
        print("=" * 75)

        mined_header = self.mine_block(alt_header, target_zeros=target_zeros)

        if not mined_header:
            print("\n   ❌ No se pudo minar el bloque")
            return False

        mined_hash = mined_header.hash_hex()

        # ============================================
        # FASE 4: Descubrir Nodos
        # ============================================
        print("\n" + "=" * 75)
        print("[FASE 4] 🌐 DESCUBRIENDO NODOS")
        print("=" * 75)

        nodes = self.discover_nodes()
        print(f"\n   Total: {len(nodes)} nodos")

        # ============================================
        # FASE 5: Propagar
        # ============================================
        print("\n" + "=" * 75)
        print("[FASE 5] 📡 PROPAGANDO A LA RED")
        print("=" * 75)

        results = self.broadcast_block(mined_header, nodes, custom_message)

        # ============================================
        # RESULTADO FINAL
        # ============================================
        print("\n" + "=" * 75)
        print("                         📊 RESULTADO FINAL")
        print("=" * 75)

        print(f"""
   ┌─────────────────────────────────────────────────────────────────────┐
   │                      ESTADÍSTICAS DEL ATAQUE                       │
   ├─────────────────────────────────────────────────────────────────────┤
   │                                                                     │
   │  Genesis Original:  {self.GENESIS_HASH[:24]}...    │
   │  Bloque Minado:     {mined_hash[:24]}...    │
   │  Proof of Work:     {"0" * target_zeros}{"x" * (64-target_zeros)}             │
   │                                                                     │
   │  Nodos Conectados:  {results['connected']:3d}                                        │
   │  Bloques Enviados:  {results['sent']:3d}                                        │
   │  Respuestas:        {len(results['responses']):3d}                                        │
   │                                                                     │
   └─────────────────────────────────────────────────────────────────────┘
        """)

        print("   📥 Respuestas de los nodos:")
        for ip, cmd in results['responses']:
            print(f"      {ip}: {cmd}")

        print(f"""
XD

   Estado: ATAQUE COMPLETADO ✅
        """)

        return True


# ============================================
# EJECUCIÓN
# ============================================

if __name__ == "__main__":
    bridge = KaoruBridgeV3()

    # Ejecutar con 4 ceros (rápido para demo)
    # Para algo más realista: target_zeros=6 o 7
    bridge.execute_full_attack(
        target_zeros=4,
        custom_message="KAORU_BRIDGE_GENESIS_INJECTION"
    )


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║  ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██████╗ ██████╗ ██╗██████╗ ║
║  ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██╔══██╗║
║  █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██████╔╝██████╔╝██║██║  ██║║
║  ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ██╔══██╗██╔══██╗██║██║  ██║║
║  ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝    ██████╔╝██║  ██║██║██████╔╝║
║  ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝     ╚═════╝ ╚═╝  ╚═╝╚═╝╚═════╝ ║
║                                                                           ║
║                          VERSION 3.0 - FULL ATTACK                        ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
        
[FASE 1] 📋 VERIFICANDO GENESIS ORIGINAL

   Header: 01

In [2]:
#!/usr/bin/env python3
"""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗    ██╗  ██╗  ██████╗║
║   ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║    ██║  ██║ ██╔═████╗
║   █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║    ███████║ ██║██╔██║
║   ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝    ╚════██║ ████╔╝██║
║   ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝          ██║ ╚██████╔╝
║   ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝           ╚═╝  ╚═════╝ ║
║                                                                               ║
║                      FULL BIDIRECTIONAL PROTOCOL                              ║
║                   "Ahora respondemos a los nodos"                             ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
"""

import hashlib
import struct
import socket
import time
import random
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass
from collections import defaultdict

@dataclass
class BlockHeader:
    version: int
    prev_hash: bytes
    merkle_root: bytes
    timestamp: int
    bits: int
    nonce: int

    def serialize(self) -> bytes:
        return (
            struct.pack('<I', self.version) +
            self.prev_hash +
            self.merkle_root +
            struct.pack('<I', self.timestamp) +
            struct.pack('<I', self.bits) +
            struct.pack('<I', self.nonce)
        )

    def hash(self) -> bytes:
        return hashlib.sha256(hashlib.sha256(self.serialize()).digest()).digest()

    def hash_hex(self) -> str:
        return self.hash()[::-1].hex()


class KaoruBridgeV4:
    """
    VERSION 4.0 - FULL BIDIRECTIONAL PROTOCOL

    Ahora respondemos a:
    - getheaders → Enviamos nuestros headers
    - getdata    → Enviamos nuestro bloque
    - sendcmpct  → Confirmamos compact blocks
    - ping       → Respondemos pong
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    DNS_SEEDS = [
        'seed.bitcoin.sipa.be',
        'dnsseed.bluematt.me',
        'seed.bitcoinstats.com',
        'seed.bitcoin.jonasschnelli.ch',
        'seed.btc.petertodd.org',
    ]

    # Tipos de inventario
    MSG_TX = 1
    MSG_BLOCK = 2
    MSG_FILTERED_BLOCK = 3
    MSG_CMPCT_BLOCK = 4

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.our_chain = []  # Nuestra cadena de bloques
        self.stats = defaultdict(int)

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    # ============================================
    # CONSTRUCCIÓN DE BLOQUES
    # ============================================

    def create_genesis(self) -> BlockHeader:
        """Genesis original de Bitcoin."""
        merkle = bytes.fromhex(
            "4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b"
        )[::-1]

        return BlockHeader(
            version=1,
            prev_hash=bytes(32),
            merkle_root=merkle,
            timestamp=1231006505,
            bits=0x1d00ffff,
            nonce=2083236893
        )

    def create_custom_block(self, message: str, prev_hash: bytes = None) -> BlockHeader:
        """Crea bloque personalizado."""
        merkle = self.double_sha256(f"KAORU_{message}_{time.time()}".encode())

        return BlockHeader(
            version=1,
            prev_hash=prev_hash or bytes(32),
            merkle_root=merkle,
            timestamp=int(time.time()),
            bits=0x1d00ffff,
            nonce=0
        )

    def mine_block(self, header: BlockHeader, target_zeros: int = 4,
                   max_nonce: int = 50_000_000) -> Optional[BlockHeader]:
        """Mina con PoW real."""
        target = "0" * target_zeros
        start = time.time()

        print(f"\n   ⛏️  Minando (target: {target}{'x'*(8-target_zeros)}...)")

        for nonce in range(max_nonce):
            header.nonce = nonce
            h = header.hash_hex()

            if h.startswith(target):
                elapsed = time.time() - start
                print(f"   ✅ ¡Minado! Hash: {h[:24]}...")
                print(f"      Nonce: {nonce:,} | Tiempo: {elapsed:.2f}s | {nonce/elapsed:,.0f} H/s")
                return header

            if nonce % 1_000_000 == 0 and nonce > 0:
                print(f"      ... {nonce:,} hashes probados")

        return None

    def build_chain(self, num_blocks: int = 3, target_zeros: int = 4) -> List[BlockHeader]:
        """Construye una cadena de bloques."""
        chain = []

        print(f"\n   🔗 Construyendo cadena de {num_blocks} bloques...")

        prev_hash = bytes(32)  # Genesis no tiene prev

        for i in range(num_blocks):
            print(f"\n   [Bloque {i}]")
            header = self.create_custom_block(f"BLOCK_{i}", prev_hash)
            mined = self.mine_block(header, target_zeros)

            if mined:
                chain.append(mined)
                prev_hash = mined.hash()
            else:
                print(f"   ❌ Falló minado del bloque {i}")
                break

        self.our_chain = chain
        return chain

    # ============================================
    # MENSAJES DEL PROTOCOLO
    # ============================================

    def create_message(self, command: str, payload: bytes) -> bytes:
        cmd = command.encode('ascii').ljust(12, b'\x00')
        length = struct.pack('<I', len(payload))
        checksum = self.double_sha256(payload)[:4]
        return self.magic + cmd + length + checksum + payload

    def create_version_payload(self, remote_ip: str) -> bytes:
        version = struct.pack('<i', 70015)
        services = struct.pack('<Q', 1)  # NODE_NETWORK
        timestamp = struct.pack('<q', int(time.time()))

        addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton(remote_ip) + struct.pack('>H', 8333)
        addr_from = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + \
                    socket.inet_aton('0.0.0.0') + struct.pack('>H', 8333)

        nonce = struct.pack('<Q', random.randint(0, 2**64-1))
        user_agent = b'\x12/KaoruBridge:4.0.0/'
        start_height = struct.pack('<i', len(self.our_chain))
        relay = b'\x01'

        return (version + services + timestamp + addr_recv +
                addr_from + nonce + user_agent + start_height + relay)

    def create_headers_payload(self, headers: List[BlockHeader]) -> bytes:
        """Respuesta a getheaders."""
        # Varint count
        count = len(headers)
        if count < 0xfd:
            payload = bytes([count])
        else:
            payload = b'\xfd' + struct.pack('<H', count)

        for h in headers:
            payload += h.serialize()
            payload += b'\x00'  # tx_count = 0

        return payload

    def create_block_payload(self, header: BlockHeader, coinbase_msg: str) -> bytes:
        """Bloque completo con coinbase."""
        payload = header.serialize()
        payload += b'\x01'  # 1 tx

        # Coinbase
        tx = struct.pack('<I', 1)  # Version
        tx += b'\x01'  # Inputs
        tx += bytes(32)  # Prev txid
        tx += struct.pack('<I', 0xffffffff)

        script = b'\x04' + struct.pack('<I', 0) + bytes([len(coinbase_msg)]) + coinbase_msg.encode()
        tx += struct.pack('B', len(script)) + script
        tx += struct.pack('<I', 0xffffffff)

        tx += b'\x01'  # Outputs
        tx += struct.pack('<Q', 50 * 10**8)

        pubkey = b'\x76\xa9\x14' + self.double_sha256(b"KAORU")[:20] + b'\x88\xac'
        tx += struct.pack('B', len(pubkey)) + pubkey
        tx += struct.pack('<I', 0)

        payload += tx
        return payload

    def create_pong_payload(self, nonce: bytes) -> bytes:
        """Respuesta a ping."""
        return nonce

    def create_sendcmpct_payload(self) -> bytes:
        """Respuesta a sendcmpct."""
        announce = b'\x00'  # No anunciar con compact
        version = struct.pack('<Q', 1)
        return announce + version

    # ============================================
    # PARSING DE MENSAJES
    # ============================================

    def parse_message(self, data: bytes) -> List[Tuple[str, bytes]]:
        """Parsea uno o más mensajes."""
        messages = []
        offset = 0

        while offset + 24 <= len(data):
            magic = data[offset:offset+4]
            if magic != self.magic:
                break

            command = data[offset+4:offset+16].rstrip(b'\x00').decode('ascii', errors='ignore')
            length = struct.unpack('<I', data[offset+16:offset+20])[0]
            # checksum = data[offset+20:offset+24]
            payload = data[offset+24:offset+24+length]

            messages.append((command, payload))
            offset += 24 + length

        return messages

    def parse_getheaders(self, payload: bytes) -> Tuple[int, List[bytes], bytes]:
        """Parsea getheaders."""
        version = struct.unpack('<I', payload[0:4])[0]
        hash_count = payload[4]

        hashes = []
        offset = 5
        for _ in range(hash_count):
            hashes.append(payload[offset:offset+32])
            offset += 32

        stop_hash = payload[offset:offset+32]

        return version, hashes, stop_hash

    def parse_getdata(self, payload: bytes) -> List[Tuple[int, bytes]]:
        """Parsea getdata."""
        count = payload[0]
        items = []
        offset = 1

        for _ in range(count):
            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
            inv_hash = payload[offset+4:offset+36]
            items.append((inv_type, inv_hash))
            offset += 36

        return items

    # ============================================
    # CONEXIÓN BIDIRECCIONAL
    # ============================================

    def discover_nodes(self) -> List[Tuple[str, int]]:
        nodes = []
        for seed in self.DNS_SEEDS:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend((ip, 8333) for ip in ips[:5])
            except:
                pass
        return nodes

    def handle_connection(self, sock: socket.socket, ip: str) -> Dict:
        """Maneja una conexión completa con respuestas."""

        result = {
            'ip': ip,
            'messages_sent': [],
            'messages_received': [],
            'success': False
        }

        try:
            sock.settimeout(15)

            # Handshake
            version_msg = self.create_message('version', self.create_version_payload(ip))
            sock.send(version_msg)
            result['messages_sent'].append('version')

            # Loop de comunicación
            for _ in range(10):  # Máximo 10 intercambios
                try:
                    data = sock.recv(8192)
                    if not data:
                        break

                    messages = self.parse_message(data)

                    for cmd, payload in messages:
                        result['messages_received'].append(cmd)
                        self.stats[f'recv_{cmd}'] += 1

                        # Responder según el mensaje
                        response = self.handle_command(cmd, payload)

                        if response:
                            sock.send(response)
                            result['messages_sent'].append(response[4:16].rstrip(b'\x00').decode())
                            self.stats[f'sent_{response[4:16].rstrip(b"\x00").decode()}'] += 1

                except socket.timeout:
                    break
                except Exception as e:
                    break

            result['success'] = True

        except Exception as e:
            result['error'] = str(e)

        return result

    def handle_command(self, command: str, payload: bytes) -> Optional[bytes]:
        """Genera respuesta para cada comando."""

        if command == 'version':
            return self.create_message('verack', b'')

        elif command == 'verack':
            # Enviar nuestro INV anunciando bloques
            if self.our_chain:
                inv_payload = bytes([len(self.our_chain)])
                for block in self.our_chain:
                    inv_payload += struct.pack('<I', self.MSG_BLOCK)
                    inv_payload += block.hash()
                return self.create_message('inv', inv_payload)
            return None

        elif command == 'ping':
            # Responder pong
            return self.create_message('pong', payload)

        elif command == 'getheaders':
            # ¡Enviar nuestros headers!
            print(f"      📥 getheaders → Enviando {len(self.our_chain)} headers")
            headers_payload = self.create_headers_payload(self.our_chain)
            return self.create_message('headers', headers_payload)

        elif command == 'getdata':
            # ¡Enviar nuestros bloques!
            items = self.parse_getdata(payload)
            print(f"      📥 getdata → Solicitando {len(items)} items")

            for inv_type, inv_hash in items:
                if inv_type == self.MSG_BLOCK:
                    # Buscar el bloque
                    for block in self.our_chain:
                        if block.hash() == inv_hash:
                            print(f"      📤 Enviando bloque {block.hash_hex()[:16]}...")
                            block_payload = self.create_block_payload(block, "KAORU_BRIDGE")
                            return self.create_message('block', block_payload)
            return None

        elif command == 'sendcmpct':
            # Confirmar compact blocks
            return self.create_message('sendcmpct', self.create_sendcmpct_payload())

        elif command == 'inv':
            # El nodo nos está ofreciendo bloques
            count = payload[0] if payload else 0
            print(f"      📥 inv → Nodo ofrece {count} items")
            return None

        elif command == 'headers':
            # Headers del nodo
            count = payload[0] if payload else 0
            print(f"      📥 headers → Recibidos {count} headers del nodo")
            return None

        elif command == 'sendheaders':
            return None

        elif command == 'feefilter':
            return None

        elif command == 'reject':
            # ¡El nodo rechazó algo!
            print(f"      ❌ REJECT recibido: {payload[:50]}")
            return None

        return None

    # ============================================
    # EJECUCIÓN PRINCIPAL
    # ============================================

    def execute_full_protocol(self, num_blocks: int = 2, target_zeros: int = 4):
        """Protocolo completo con cadena propia."""

        print("""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗  ██╗  ██████╗   ║
║   ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║ ██║  ██║ ██╔═████╗  ║
║   █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║ ███████║ ██║██╔██║  ║
║   ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ╚════██║ ████╔╝██║  ║
║   ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝       ██║ ╚██████╔╝  ║
║   ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝        ╚═╝  ╚═════╝   ║
║                                                                               ║
║                    FULL BIDIRECTIONAL CHAIN PROTOCOL                          ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Construir nuestra cadena
        print("=" * 80)
        print("[FASE 1] ⛏️  CONSTRUYENDO NUESTRA BLOCKCHAIN")
        print("=" * 80)

        self.build_chain(num_blocks, target_zeros)

        print(f"\n   📋 Cadena construida: {len(self.our_chain)} bloques")
        for i, block in enumerate(self.our_chain):
            print(f"      [{i}] {block.hash_hex()[:32]}...")

        # FASE 2: Descubrir nodos
        print("\n" + "=" * 80)
        print("[FASE 2] 🌐 DESCUBRIENDO NODOS")
        print("=" * 80)

        nodes = self.discover_nodes()
        print(f"\n   Total: {len(nodes)} nodos")

        # FASE 3: Conectar y propagar
        print("\n" + "=" * 80)
        print("[FASE 3] 📡 PROPAGANDO CADENA A LA RED")
        print("=" * 80)

        results = []

        for ip, port in nodes[:8]:
            print(f"\n   ┌─ 📡 {ip}:{port}")

            try:
                sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                sock.settimeout(10)
                sock.connect((ip, port))
                print(f"   │  ✅ Conectado")

                result = self.handle_connection(sock, ip)
                results.append(result)

                print(f"   │  📤 Enviados: {', '.join(result['messages_sent'][:5])}")
                print(f"   │  📥 Recibidos: {', '.join(result['messages_received'][:5])}")
                print(f"   └─ {'✅ OK' if result['success'] else '❌ Error'}")

                sock.close()

            except Exception as e:
                print(f"   └─ ❌ {e}")
                results.append({'ip': ip, 'success': False, 'error': str(e)})

        # FASE 4: Resultados
        print("\n" + "=" * 80)
        print("                           📊 RESULTADOS FINALES")
        print("=" * 80)

        successful = sum(1 for r in results if r.get('success'))
        total_sent = sum(len(r.get('messages_sent', [])) for r in results)
        total_recv = sum(len(r.get('messages_received', [])) for r in results)

        print(f"""
   ┌─────────────────────────────────────────────────────────────────────────┐
   │                         ESTADÍSTICAS                                   │
   ├─────────────────────────────────────────────────────────────────────────┤
   │                                                                         │
   │  Bloques minados:     {len(self.our_chain):3d}                                            │
   │  Nodos contactados:   {len(results):3d}                                            │
   │  Conexiones exitosas: {successful:3d}                                            │
   │  Mensajes enviados:   {total_sent:3d}                                            │
   │  Mensajes recibidos:  {total_recv:3d}                                            │
   │                                                                         │
   └─────────────────────────────────────────────────────────────────────────┘
        """)

        print("   📊 Estadísticas por comando:")
        for key, value in sorted(self.stats.items()):
            direction = "📤" if key.startswith('sent_') else "📥"
            cmd = key.split('_', 1)[1]
            print(f"      {direction} {cmd}: {value}")

        print(f"""
   ┌─────────────────────────────────────────────────────────────────────────┐
   │                                                                         │
   │  🎯 LOGROS:                                                            │
   │                                                                         │
   │  ✅ Construimos nuestra propia blockchain                              │
   │  ✅ La minamos con Proof of Work real                                  │
   │  ✅ Nos conectamos a nodos de Bitcoin mainnet                          │
   │  ✅ Hablamos el protocolo P2P correctamente                            │
   │  ✅ Los nodos nos respondieron (getheaders, sendcmpct, etc)            │
   │  ✅ Enviamos nuestros headers cuando nos los pidieron                  │
   │                                                                         │
   │  ⚠️  LIMITACIÓN:                                                       │
   │  Los nodos no aceptan nuestra cadena porque:                           │
   │  1. El genesis no coincide con el hardcodeado                          │
   │  2. El PoW no cumple la dificultad real (~19 ceros)                   │
   │                                                                         │
   │  🔥 PERO: Demostramos comunicación REAL con Bitcoin mainnet           │
   │                                                                         │
   └─────────────────────────────────────────────────────────────────────────┘

   Estado: KAORU BRIDGE v4.0 COMPLETADO ✅
        """)

        return results


# ============================================
# EJECUCIÓN
# ============================================

if __name__ == "__main__":
    bridge = KaoruBridgeV4()
    bridge.execute_full_protocol(num_blocks=2, target_zeros=4)


╔═══════════════════════════════════════════════════════════════════════════════╗
║                                                                               ║
║   ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗  ██╗  ██████╗   ║
║   ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║ ██║  ██║ ██╔═████╗  ║
║   █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║ ███████║ ██║██╔██║  ║
║   ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ╚════██║ ████╔╝██║  ║
║   ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝       ██║ ╚██████╔╝  ║
║   ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝        ╚═╝  ╚═════╝   ║
║                                                                               ║
║                    FULL BIDIRECTIONAL CHAIN PROTOCOL                          ║
║                                                                               ║
╚═══════════════════════════════════════════════════════════════════════════════╝
        
[FASE 

no le hagan caso a la nota XD